In [2]:
import requests
import polyline
import turtle

In [4]:
auth_token = '<follow README to get this token>'
client_id = '<YOUR CLIENT ID>''
client_secret = '<YOUR CLIENT SECRET>'

params = {'client_id': client_id, 
          'client_secret': client_secret, 
          'code': auth_token,
          'grant_type': 'authorization_code'}

response = requests.post('https://www.strava.com/oauth/token', params=params)

In [6]:
url = 'https://www.strava.com/api/v3/athlete/activities'

bearer_token = response.json()['access_token']

# Add additional filtering parameters here.
# before = <unix timestamp>
# after = <unix timestamp>
# per_page = <int>

# Uncomment this line if you added filtering parameters above
# params = {'before': f'{before}', 'after': f'{after}', 'per_page': f'{per_page}'}

header = {'Authorization': f"Bearer {bearer_token}"}

activities = requests.get(url, headers=header, params=params).json()

In [28]:
longlats = [polyline.decode(activity['map']['summary_polyline']) for activity in activities if activity['map']['summary_polyline']]


In [30]:
t = turtle.Turtle()
t.pensize(2)
turtle.bgcolor("white")
turtle.hideturtle()
turtle.speed('fast')

colors = ["#FBEBB4", "#85A7E8", "#F0D5CB", "#C7BCE5", "#C2E3AE"]
cities = {}
scale = 1e4

for longlat in longlats:
    city = (round(longlat[0][0]), round(longlat[0][1]))
    if city not in cities.keys():
        cities[city] = [longlat[0], colors[len(cities) % len(colors)]]
    zero = cities[city][0]
    t.pencolor(cities[city][1])

    scaled = [((lat-zero[1])*scale, (long-zero[0])*scale) for long, lat in longlat]
    
    t.goto(scaled[0])
    t.pendown()
    for x, y in scaled:
        t.goto(x,y)
    t.penup()
    t.goto(0,0)